# Log2 Fold-Change example

Data have different distributions and quantitative features. Data may need to be scaled or normalised in order to be compared in a way that makes sense.

This is true for data about abundance of biological molecules, like mRNAs and proteins. Some mRNAs are very abundant and may have 1000s of copies per cell, others are rare and may have an average of less than one copy per cell. Plotting these on the same axes isn't informative.

In [ ]:
# Analysis modules
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(precision=5, suppress=True)  # suppress scientific floatation 
sns.set(color_codes=True)
%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Small Log2 Fold-Change example

This uses a very small gene expression dataset, `/Datasets/Infection_TPM.csv`, to illustrate how to calculate a log2 fold-change.

In [ ]:
df = pd.read_csv("../Datasets/Infection_TPM.csv")
df

These data aren't tidy. In practice, you could **either** first calculate the log2 fold-change and then tidy it up, **or** you could first tidy it up and then calculate. Here we will calculate first, and leave the tidy then calculate as an exercise.

## Example: calculate log2 fold-change first

In [ ]:
# remove the Gene_ID column to leave only the numeric columns
df_numbersonly = df.loc[:, df.columns != "Gene_ID"] 
df_numbersonly

In [ ]:
# calculate log2 values
df_numbersonly_log2 = np.log2(df_numbersonly)
df_numbersonly_log2

In [ ]:
# subtract the mean values of columns
df_numbersonly_log2foldchange = df_numbersonly_log2.apply(lambda x : x - x.mean(), axis = 1)
df_numbersonly_log2foldchange


In [ ]:
# check that rows sum to zero, so we calculated correctly for each gene
df_numbersonly_log2foldchange.sum(axis = 1)

In [ ]:
# concatenate the GeneID column, in case it's needed
df_log2foldchange = pd.concat([ df[['Gene_ID']], df_numbersonly_log2foldchange], axis = 1)
df_log2foldchange

Let's plot this as a heatmap. We have to set an index to get the gene names in the rowl

In [ ]:
df_numbersonly_log2foldchange.index =  df['Gene_ID']

sns.heatmap(data = df_numbersonly_log2foldchange)

This step-by-step approach works.

In order, we:

1. Extracted numeric columns
2. Calculated the log2 values
3. Subtracted the log2-means from the values
4. Concatenated (restored) the other columns
5. Checked everything as we went along.

## Calculate more efficiently using a function

Instead of writing out all the steps on the dataframe, it's a better idea to write a function.

This means that all the calculation, and all the thinking about the calculation, is wrapped up into a single piece of code, and then afterwards you use that piece again and again.

You use functions all the time in this course and you have come across functions in previous courses. It's useful, sometimes, to write your own.

We will call the function `log2foldchange`. It will take an input vector that we'll call `x`, and return the log2 fold-change of the input.

In [ ]:
def log2foldchange(x):
    x_log2 = np.log2(x)
    return x_log2 - x_log2.mean()

Let's test the function. To test the function, we'll use a simple input that the vector `[1, 2, 4]` has log2-scale values `[0, 1, 2]`, so the fold-change should be `[-1, 0, 1]`.

In [ ]:
log2foldchange(np.array([1, 2, 4]))

That worked as expected. These kind of little tests and sanity checks are important for knowing that your code works.

## Exercise 1: Use `log2foldchange` to re-calculate on the columns of the data

Try using `apply`.

## Exercise 2: Use `log2foldchange` on tidy data

Try:
1. Tidy your data using `melt`.  Give sensible names to the column with TPMs and the column with sample names.
2. Group by Gene using `group_by`.
3. Then use `log2foldchange` on the grouped TPM column.
4. Check that it worked!

## Exercise 3: Can you do `log10foldchange`? Or a different normalisation?

Different normalisations work for different purposes, depending on how spread out the data are.